In [ ]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig
import torch
import torch.nn as nn
import pandas as pd
import torch
from transformers import EsmTokenizer, EsmModel, Trainer, TrainingArguments
from accelerate import Accelerator
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
from sklearn.preprocessing import StandardScaler
from esm.tokenization import get_esmc_model_tokenizers
import os

'''
args:
'''
ESMC_path = "/home/users/hcdai/AI-peptide/Seq2Score/Henya/weights/20250302_baseline/ESMC_path/esmc_model.pth"
model_path = "/home/users/hcdai/AI-peptide/Seq2Score/Henya/weights/20250302_baseline/model_path/model.pth"



def ESMC_600M_202412(device: torch.device | str = "cpu", use_flash_attn: bool = True, model_path: str = None):
    with torch.device(device):
        model = ESMC(
            d_model=1152,
            n_heads=18,
            n_layers=36,
            tokenizer=get_esmc_model_tokenizers(),
            use_flash_attn=use_flash_attn,
        ).eval()
    if model_path is not None:
        state_dict = torch.load(
            model_path,
            map_location=device,
            weights_only=True,
        )
        # 去除 "module." 前缀
        new_state_dict = {}
        for key, value in state_dict.items():
            if key.startswith('module.'):
                new_state_dict[key[7:]] = value
            else:
                new_state_dict[key] = value
        model.load_state_dict(new_state_dict)
    else:
        print(f"Error: The model file at {model_path} was not found.")
    return model

ESMC_model = ESMC_600M_202412(device = 'cuda', use_flash_attn= True,model_path = ESMC_path)
ESMC_model.eval()

def ESMC_encoding(sequence:str):
    '''
    调用ESMC进行Encoding，但是不进行计算，用途是输出forward方法可以接受的张量
    '''
    protein = ESMProtein(sequence = sequence)
    protein_tensor = ESMC_model.encode(protein)
    return protein_tensor

class Probability_Prediction_Net(nn.Module):
    def __init__(self, 
                 input_dim=2048, 
                 num_heads=8, 
                 num_blocks=12, 
                 ESMC_d_model = 1152,
                 ESMC = ESMC_model):
        super(Probability_Prediction_Net, self).__init__()
        self.input_dim = input_dim
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.Linear = nn.Sequential(   
            nn.Linear(ESMC_d_model, input_dim*4),
            nn.ReLU(),
            nn.Dropout(p = 0.4),
            nn.Linear(input_dim*4, input_dim*4),
            nn.ReLU(),
            nn.Dropout(p = 0.4),
            nn.Linear(input_dim*4, input_dim)     
        )
        # self.ESMC = ESMC

        # 定义 Transformer 编码器层
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, 
                                                   nhead=num_heads, 
                                                   dim_feedforward=input_dim * 4,
                                                   activation = "relu",
                                                   batch_first = True,
                                                   norm_first = True)
        # 定义 Transformer 编码器
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_blocks) 
        self.fc1 = nn.Sequential(   
            nn.Linear(input_dim, input_dim*4),
            nn.BatchNorm1d(input_dim*4), 
            nn.ReLU(), 
            nn.Dropout(p = 0.4),
            nn.Linear(input_dim*4, input_dim*4),
            nn.ReLU(),
            nn.BatchNorm1d(input_dim*4), 
            nn.Dropout(p = 0.4),
            nn.Linear(input_dim*4, 2)   
        )

    def forward(self, merge, label=None):
        # merge = self.ESMC.logits(merge,LogitsConfig(sequence = True,return_hidden_states = True))
        # merge = merge.hidden_states[-1, :, :, :].shape
        
        x = self.Linear(merge)
        # 调整输入形状以适应 Transformer 编码器的输入要求 [seq_len, batch_size, input_dim]
        batch_size, seq_len, _ = x.shape
        x = x.view(batch_size, seq_len, -1)

        # 通过 Transformer 编码器
        x = self.transformer_encoder(x)
        x = x[ :, 0, :]
        x = self.fc1(x)
        

        if label is not None:
            loss_fn = torch.nn.BCEWithLogitsLoss()# 使用二元交叉熵损失函数
            loss = loss_fn(x, label)
            return loss, x
        else:
            return x
        
def load_model_from_checkpoint(device: torch.device | str = "cpu",checkpoint_path=None):
    '''
    加载概率模型的函数
    '''
    model = Probability_Prediction_Net()
    if checkpoint_path:
        state_dict = torch.load(checkpoint_path, map_location=device)
        # 移除 'module.' 前缀
        new_state_dict = {}
        for key, value in state_dict.items():
            if key.startswith('module.'):
                new_key = key[7:]  # 移除 'module.'
                new_state_dict[new_key] = value
            else:
                new_state_dict[key] = value
        model.load_state_dict(new_state_dict)
    return model.to(device)
    
model = load_model_from_checkpoint(device = 'cuda',checkpoint_path = model_path)
model.eval()

seq1 = 'NLVPMVATV'
seq2 = 'CSAKGGQHPYEQYF'
merge = seq1 + "|" + seq2
merge = ESMC_encoding(merge)
merge = ESMC_model.logits(merge,LogitsConfig(sequence = True,return_hidden_states = True))
merge = merge.hidden_states[-1, :, :, :]
print(merge.shape)
sigmoid = nn.Sigmoid()
merge = sigmoid(model(merge = merge, label = None))
print(merge)





/home/users/hcdai/miniconda3/envs/ESMC/lib/python3.11/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
/tmp/ipykernel_384404/1002392119.py:133: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` 

Probability_Prediction_Net(
  (Linear): Sequential(
    (0): Linear(in_features=1152, out_features=8192, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=8192, out_features=8192, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.4, inplace=False)
    (6): Linear(in_features=8192, out_features=2048, bias=True)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=2048, out_features=2048, bias=True)
        )
        (linear1): Linear(in_features=2048, out_features=8192, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=8192, out_features=2048, bias=True)
        (norm1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1

In [2]:
input_df_path = '/home/users/hcdai/AI-peptide/Seq2Score/Henya/dataset/panpep_test/finetuning_test/for_compare.csv'
input_df = pd.read_csv(input_df_path)
output_path = '/home/users/hcdai/AI-peptide/Seq2Score/Henya/dataset/panpep_test/finetuning_test/for_compare_prob_not_finetuned.csv'
from tqdm import tqdm

probability_positive = []
probability_negative = []
for index, row in tqdm(input_df.iterrows(), total=len(input_df), desc="Processing PDB files"):
    receptor = row['binding_TCR']
    peptide = row['peptide']
    merge = receptor + "|" + peptide
    merge = ESMC_encoding(merge)
    merge = ESMC_model.logits(merge,LogitsConfig(sequence = True,return_hidden_states = True))
    merge = merge.hidden_states[-1, :, :, :]
    sigmoid = nn.Sigmoid()
    merge = sigmoid(model(merge = merge, label = None))
    merge = merge.cpu().detach().numpy()
    print(merge)
    probability_positive.append(merge[0][0])
    probability_negative.append(merge[0][1])
    del merge,receptor,peptide

input_df['Probability_positive'] = probability_positive
input_df['Probability_negative'] = probability_negative
input_df.to_csv(output_path)

Processing PDB files:   0%|          | 0/4184 [00:00<?, ?it/s]

Processing PDB files:   0%|          | 4/4184 [00:00<06:01, 11.55it/s]

[[0.45368946 0.54052335]]
[[0.28759307 0.7136908 ]]
[[0.35733137 0.64623576]]
[[0.30450162 0.68678284]]
[[0.5907668 0.383397 ]]


Processing PDB files:   0%|          | 10/4184 [00:00<03:47, 18.31it/s]

[[0.3417351  0.64484185]]
[[0.37291852 0.6080964 ]]
[[0.23508841 0.75035816]]
[[0.32261327 0.67743534]]
[[0.23180772 0.74760824]]


...

[[0.55377245 0.4401758 ]]
[[0.45347518 0.56479615]]
[[0.3460438  0.64489627]]
[[0.5976188  0.38424572]]
[[0.3340802  0.66004103]]


Processing PDB files: 100%|█████████▉| 4177/4184 [02:56<00:00, 24.05it/s]

[[0.2884608 0.7059901]]
[[0.30311498 0.6774957 ]]
[[0.49149796 0.5081255 ]]
[[0.23853956 0.75767386]]
[[0.5444513  0.45175695]]


Processing PDB files: 100%|██████████| 4184/4184 [02:56<00:00, 23.65it/s]

[[0.44939643 0.5559229 ]]
[[0.34128875 0.6510997 ]]
[[0.31764254 0.67913973]]
[[0.3453374 0.6397203]]
[[0.337309  0.6534594]]


In [ ]:
input_df = pd.read_csv('/home/users/hcdai/AI-peptide/Seq2Score/Henya/dataset/panpep_test/finetuning_test/Majority_from_panpep_test.csv')

train_df, valid_df = train_test_split(input_df, test_size=0.2, random_state=42)
# 重置索引，保证索引连续，避免出现KeyError
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
train_df.to_csv('/home/users/hcdai/AI-peptide/Seq2Score/Henya/dataset/panpep_test/finetuning_test/train.csv', index=False)
valid_df.to_csv('/home/users/hcdai/AI-peptide/Seq2Score/Henya/dataset/panpep_test/finetuning_test/test.csv', index=False)

: 

# Finetune test
- 把majority from panpep拿过来用
- 用原型正确率60
- 分离20%用来做小样本学习，训练模型，参数为：
  - '''training_args = TrainingArguments(
    output_dir= output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir= logging_dir,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    fp16=True,  # 启用混合精度训练
    )   '''
  - random = 1
